In [6]:
%load_ext autoreload

%autoreload 2



import torch
from vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor
from vision.datasets.voc_dataset import VOCDataset

from vision.utils import box_utils, measurements
from vision.utils.misc import str2bool, Timer
import argparse
import pathlib
import numpy as np
import logging
import sys


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


In [8]:
def group_annotation_by_class(dataset):
    true_case_stat = {}
    all_gt_boxes = {}
    all_difficult_cases = {}
    for i in range(len(dataset)):
        image_id, annotation = dataset.get_annotation(i)
        gt_boxes, classes, is_difficult = annotation
        gt_boxes = torch.from_numpy(gt_boxes)
        for i, difficult in enumerate(is_difficult):
            class_index = int(classes[i])
            gt_box = gt_boxes[i]
            if not difficult:
                true_case_stat[class_index] = true_case_stat.get(class_index, 0) + 1

            if class_index not in all_gt_boxes:
                all_gt_boxes[class_index] = {}
            if image_id not in all_gt_boxes[class_index]:
                all_gt_boxes[class_index][image_id] = []
            all_gt_boxes[class_index][image_id].append(gt_box)
            if class_index not in all_difficult_cases:
                all_difficult_cases[class_index]={}
            if image_id not in all_difficult_cases[class_index]:
                all_difficult_cases[class_index][image_id] = []
            all_difficult_cases[class_index][image_id].append(difficult)

    for class_index in all_gt_boxes:
        for image_id in all_gt_boxes[class_index]:
            all_gt_boxes[class_index][image_id] = torch.stack(all_gt_boxes[class_index][image_id])
    for class_index in all_difficult_cases:
        for image_id in all_difficult_cases[class_index]:
            all_gt_boxes[class_index][image_id] = torch.tensor(all_gt_boxes[class_index][image_id])
    return true_case_stat, all_gt_boxes, all_difficult_cases



In [9]:

def compute_average_precision_per_class(num_true_cases, gt_boxes, difficult_cases,
                                        prediction_file, iou_threshold, use_2007_metric):
    with open(prediction_file) as f:
        image_ids = []
        boxes = []
        scores = []
        for line in f:
            t = line.rstrip().split(" ")
            image_ids.append(t[0])
            scores.append(float(t[1]))
            box = torch.tensor([float(v) for v in t[2:]]).unsqueeze(0)
            box -= 1.0  # convert to python format where indexes start from 0
            boxes.append(box)
        scores = np.array(scores)
        sorted_indexes = np.argsort(-scores)
        boxes = [boxes[i] for i in sorted_indexes]
        image_ids = [image_ids[i] for i in sorted_indexes]
        true_positive = np.zeros(len(image_ids))
        false_positive = np.zeros(len(image_ids))
        matched = set()
        for i, image_id in enumerate(image_ids):
            box = boxes[i]
            if image_id not in gt_boxes:
                false_positive[i] = 1
                continue

            gt_box = gt_boxes[image_id]
            ious = box_utils.iou_of(box, gt_box)
            max_iou = torch.max(ious).item()
            max_arg = torch.argmax(ious).item()
            if max_iou > iou_threshold:
                if difficult_cases[image_id][max_arg] == 0:
                    if (image_id, max_arg) not in matched:
                        true_positive[i] = 1
                        matched.add((image_id, max_arg))
                    else:
                        false_positive[i] = 1
            else:
                false_positive[i] = 1

    true_positive = true_positive.cumsum()
    false_positive = false_positive.cumsum()
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / num_true_cases
    if use_2007_metric:
        return measurements.compute_voc2007_average_precision(precision, recall)
    else:
        return measurements.compute_average_precision(precision, recall)



In [10]:


dataset_type = "voc"
use_cuda=True
use_2007_metric=True
nms_method="hard"
iou_threshold=0.5
mb2_width_mult=1.0
eval_dir = "results_vgg16_eval_testset"
#dataset = "~/data/VOC0712/VOC2012/"
dataset = "/datasets/home/12/312/vpotnuru/data/VOC0712/VOC2007/" #point it to the directory after downloading the dataset 
label_file = "models/voc-model-labels.txt"
trained_model = "models/vgg16-ssd-mp-0_7726.pth"
eval_mode = "test"


In [11]:



eval_path = pathlib.Path(eval_dir) #
eval_path.mkdir(exist_ok=True)
timer = Timer()
class_names = [name.strip() for name in open(label_file).readlines()]

dataset = VOCDataset(dataset, is_test=eval_mode ) #is_test just signifies dataset type here : False: train dataset ; True: test dataset

true_case_stat, all_gb_boxes, all_difficult_cases = group_annotation_by_class(dataset)

net = create_vgg_ssd(len(class_names), is_test=True)


timer.start("Load Model")
net.load(trained_model)
net = net.to(DEVICE)
print(f'It took {timer.end("Load Model")} seconds to load the model.')

predictor = create_vgg_ssd_predictor(net, nms_method=nms_method, device=DEVICE)

results = []
for i in range(len(dataset)):
    print("process image", i)
    timer.start("Load Image")
    image = dataset.get_image(i)
    print("Load Image: {:4f} seconds.".format(timer.end("Load Image")))
    timer.start("Predict")
    boxes, labels, probs = predictor.predict(image)
    print("Prediction: {:4f} seconds.".format(timer.end("Predict")))
    indexes = torch.ones(labels.size(0), 1, dtype=torch.float32) * i
    results.append(torch.cat([
        indexes.reshape(-1, 1),
        labels.reshape(-1, 1).float(),
        probs.reshape(-1, 1),
        boxes + 1.0  # matlab's indexes start from 1
    ], dim=1))
results = torch.cat(results)
for class_index, class_name in enumerate(class_names):
    if class_index == 0: continue  # ignore background
    prediction_path = eval_path / f"det_test_{class_name}.txt"
    with open(prediction_path, "w") as f:
        sub = results[results[:, 1] == class_index, :]
        for i in range(sub.size(0)):
            prob_box = sub[i, 2:].numpy()
            image_id = dataset.ids[int(sub[i, 0])]
            print(
                image_id + " " + " ".join([str(v) for v in prob_box]),
                file=f
            )
aps = []
print("\n\nAverage Precision Per-class:")
for class_index, class_name in enumerate(class_names):
    if class_index == 0:
        continue
    prediction_path = eval_path / f"det_test_{class_name}.txt"
    ap = compute_average_precision_per_class(
        true_case_stat[class_index],
        all_gb_boxes[class_index],
        all_difficult_cases[class_index],
        prediction_path,
        iou_threshold,
        use_2007_metric
    )
    aps.append(ap)
    print(f"{class_name}: {ap}")

print(f"\nAverage Precision Across All Classes:{sum(aps)/len(aps)}")





/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


It took 0.11566996574401855 seconds to load the model.
process image 0
Load Image: 0.014407 seconds.
Inference time:  0.31606316566467285
Prediction: 0.330831 seconds.
process image 1
Load Image: 0.004905 seconds.
Inference time:  0.03084564208984375
Prediction: 0.070095 seconds.
process image 2
Load Image: 0.004883 seconds.
Inference time:  0.0073010921478271484
Prediction: 0.195753 seconds.
process image 3
Load Image: 0.005434 seconds.
Inference time:  0.005272388458251953
Prediction: 0.085198 seconds.
process image 4
Load Image: 0.077468 seconds.
Inference time:  0.09347248077392578
Prediction: 0.121762 seconds.
process image 5
Load Image: 0.005047 seconds.
Inference time:  0.006512641906738281
Prediction: 0.187707 seconds.
process image 6
Load Image: 0.008861 seconds.
Inference time:  0.07162117958068848
Prediction: 0.095305 seconds.
process image 7
Load Image: 0.006290 seconds.
Inference time:  0.00937509536743164
Prediction: 0.180015 seconds.
process image 8
Load Image: 0.011793 

Load Image: 0.007004 seconds.
Inference time:  0.012562990188598633
Prediction: 0.225912 seconds.
process image 73
Load Image: 0.004077 seconds.
Inference time:  0.045867919921875
Prediction: 0.059314 seconds.
process image 74
Load Image: 0.007232 seconds.
Inference time:  0.008821725845336914
Prediction: 0.105741 seconds.
process image 75
Load Image: 0.084926 seconds.
Inference time:  0.008184432983398438
Prediction: 0.112936 seconds.
process image 76
Load Image: 0.007126 seconds.
Inference time:  0.009199857711791992
Prediction: 0.200361 seconds.
process image 77
Load Image: 0.005012 seconds.
Inference time:  0.004859447479248047
Prediction: 0.091773 seconds.
process image 78
Load Image: 0.006159 seconds.
Inference time:  0.010011672973632812
Prediction: 0.181209 seconds.
process image 79
Load Image: 0.006762 seconds.
Inference time:  0.07619571685791016
Prediction: 0.111493 seconds.
process image 80
Load Image: 0.010703 seconds.
Inference time:  0.010892629623413086
Prediction: 0.19

Prediction: 0.195601 seconds.
process image 144
Load Image: 0.004907 seconds.
Inference time:  0.06810808181762695
Prediction: 0.079528 seconds.
process image 145
Load Image: 0.011234 seconds.
Inference time:  0.012034893035888672
Prediction: 0.212084 seconds.
process image 146
Load Image: 0.004354 seconds.
Inference time:  0.04613661766052246
Prediction: 0.056444 seconds.
process image 147
Load Image: 0.006393 seconds.
Inference time:  0.09738802909851074
Prediction: 0.242452 seconds.
process image 148
Load Image: 0.006510 seconds.
Inference time:  0.03670191764831543
Prediction: 0.054451 seconds.
process image 149
Load Image: 0.084411 seconds.
Inference time:  0.01217198371887207
Prediction: 0.137942 seconds.
process image 150
Load Image: 0.006099 seconds.
Inference time:  0.05910634994506836
Prediction: 0.116690 seconds.
process image 151
Load Image: 0.004879 seconds.
Inference time:  0.009146451950073242
Prediction: 0.146342 seconds.
process image 152
Load Image: 0.009061 seconds.


Inference time:  0.013574838638305664
Prediction: 0.196970 seconds.
process image 216
Load Image: 0.004987 seconds.
Inference time:  0.004426717758178711
Prediction: 0.071923 seconds.
process image 217
Load Image: 0.005208 seconds.
Inference time:  0.013042688369750977
Prediction: 0.181785 seconds.
process image 218
Load Image: 0.011058 seconds.
Inference time:  0.0775156021118164
Prediction: 0.114029 seconds.
process image 219
Load Image: 0.005814 seconds.
Inference time:  0.014024496078491211
Prediction: 0.181413 seconds.
process image 220
Load Image: 0.007467 seconds.
Inference time:  0.0063898563385009766
Prediction: 0.102349 seconds.
process image 221
Load Image: 0.008398 seconds.
Inference time:  0.01226806640625
Prediction: 0.178317 seconds.
process image 222
Load Image: 0.008313 seconds.
Inference time:  0.006011962890625
Prediction: 0.082458 seconds.
process image 223
Load Image: 0.005830 seconds.
Inference time:  0.014906644821166992
Prediction: 0.187319 seconds.
process imag

...

Inference time:  0.008960247039794922
Prediction: 0.191846 seconds.
process image 4643
Load Image: 0.006502 seconds.
Inference time:  0.06704235076904297
Prediction: 0.090705 seconds.
process image 4644
Load Image: 0.004455 seconds.
Inference time:  0.09473276138305664
Prediction: 0.182878 seconds.
process image 4645
Load Image: 0.007951 seconds.
Inference time:  0.08322381973266602
Prediction: 0.107138 seconds.
process image 4646
Load Image: 0.003925 seconds.
Inference time:  0.01221919059753418
Prediction: 0.187415 seconds.
process image 4647
Load Image: 0.005740 seconds.
Inference time:  0.013013839721679688
Prediction: 0.097610 seconds.
process image 4648
Load Image: 0.006315 seconds.
Inference time:  0.011458396911621094
Prediction: 0.193336 seconds.
process image 4649
Load Image: 0.006675 seconds.
Inference time:  0.07244086265563965
Prediction: 0.091501 seconds.
process image 4650
Load Image: 0.008101 seconds.
Inference time:  0.09719729423522949
Prediction: 0.202148 seconds.
pr

Prediction: 0.181074 seconds.
process image 4714
Load Image: 0.007749 seconds.
Inference time:  0.008532047271728516
Prediction: 0.099643 seconds.
process image 4715
Load Image: 0.007836 seconds.
Inference time:  0.07830333709716797
Prediction: 0.121693 seconds.
process image 4716
Load Image: 0.005515 seconds.
Inference time:  0.0115203857421875
Prediction: 0.154351 seconds.
process image 4717
Load Image: 0.012391 seconds.
Inference time:  0.0729682445526123
Prediction: 0.094229 seconds.
process image 4718
Load Image: 0.007301 seconds.
Inference time:  0.01316523551940918
Prediction: 0.206766 seconds.
process image 4719
Load Image: 0.006462 seconds.
Inference time:  0.05738544464111328
Prediction: 0.083759 seconds.
process image 4720
Load Image: 0.005434 seconds.
Inference time:  0.005987405776977539
Prediction: 0.177276 seconds.
process image 4721
Load Image: 0.007025 seconds.
Inference time:  0.08210015296936035
Prediction: 0.104665 seconds.
process image 4722
Load Image: 0.006534 se

Prediction: 0.204684 seconds.
process image 4785
Load Image: 0.005786 seconds.
Inference time:  0.004945993423461914
Prediction: 0.086414 seconds.
process image 4786
Load Image: 0.005583 seconds.
Inference time:  0.0896608829498291
Prediction: 0.215894 seconds.
process image 4787
Load Image: 0.004975 seconds.
Inference time:  0.04939103126525879
Prediction: 0.093807 seconds.
process image 4788
Load Image: 0.004898 seconds.
Inference time:  0.0965876579284668
Prediction: 0.189117 seconds.
process image 4789
Load Image: 0.004924 seconds.
Inference time:  0.05443930625915527
Prediction: 0.078744 seconds.
process image 4790
Load Image: 0.007425 seconds.
Inference time:  0.012285947799682617
Prediction: 0.185776 seconds.
process image 4791
Load Image: 0.008705 seconds.
Inference time:  0.08018279075622559
Prediction: 0.106490 seconds.
process image 4792
Load Image: 0.006135 seconds.
Inference time:  0.013509988784790039
Prediction: 0.164468 seconds.
process image 4793
Load Image: 0.013808 s

Prediction: 0.105650 seconds.
process image 4856
Load Image: 0.004026 seconds.
Inference time:  0.004991292953491211
Prediction: 0.091117 seconds.
process image 4857
Load Image: 0.009642 seconds.
Inference time:  0.015410661697387695
Prediction: 0.207879 seconds.
process image 4858
Load Image: 0.005363 seconds.
Inference time:  0.004948139190673828
Prediction: 0.068486 seconds.
process image 4859
Load Image: 0.006105 seconds.
Inference time:  0.013885259628295898
Prediction: 0.203995 seconds.
process image 4860
Load Image: 0.006710 seconds.
Inference time:  0.005657672882080078
Prediction: 0.089483 seconds.
process image 4861
Load Image: 0.005243 seconds.
Inference time:  0.01354527473449707
Prediction: 0.209273 seconds.
process image 4862
Load Image: 0.005878 seconds.
Inference time:  0.00463557243347168
Prediction: 0.068611 seconds.
process image 4863
Load Image: 0.005372 seconds.
Inference time:  0.09736919403076172
Prediction: 0.217046 seconds.
process image 4864
Load Image: 0.0052

Prediction: 0.191534 seconds.
process image 4928
Load Image: 0.006332 seconds.
Inference time:  0.0611879825592041
Prediction: 0.081936 seconds.
process image 4929
Load Image: 0.007191 seconds.
Inference time:  0.013710737228393555
Prediction: 0.192611 seconds.
process image 4930
Load Image: 0.012444 seconds.
Inference time:  0.008700847625732422
Prediction: 0.121584 seconds.
process image 4931
Load Image: 0.005645 seconds.
Inference time:  0.005645036697387695
Prediction: 0.083074 seconds.
process image 4932
Load Image: 0.077077 seconds.
Inference time:  0.0076160430908203125
Prediction: 0.111380 seconds.
process image 4933
Load Image: 0.008511 seconds.
Inference time:  0.012312650680541992
Prediction: 0.109458 seconds.
process image 4934
Load Image: 0.005374 seconds.
Inference time:  0.09798145294189453
Prediction: 0.192333 seconds.
process image 4935
Load Image: 0.005994 seconds.
Inference time:  0.06381916999816895
Prediction: 0.089288 seconds.
process image 4936
Load Image: 0.0046